In [6]:
install.packages('reticulate')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [7]:
library("reticulate")

In [10]:
list.files()

[1] "configNquery.ipynb"     "configNquery.py"        "configNqueryCmdline.py"
[4] "credentials.json"       "readme.md"              "rIEMifViaRest.py"      
[7] "Untitled.ipynb"

In [25]:
print(rCfg)

[1] "../credentials.json"


In [33]:
os <- import("os")
os$listdir("..")

[1] ".DS_Store"                                                                                                
 [2] ".git"                                                                                                     
 [3] ".idea"                                                                                                    
 [4] ".ipynb_checkpoints"                                                                                       
 [5] "baseline_template.bes"                                                                                    
 [6] "baselineActionReport.bes"                                                                                 
 [7] "bfClientSetting"                                                                                          
 [8] "completedBaseline.bes"                                                                                    
 [9] "configNquery"                                                                                             
[10] "createComputerGroup.py"                                                                                   
[11] "createComputerGroup2.py"                                                                                  
[12] "credentials.json"                                                                                         
[13] "cycleGather.py"                                                                                           
[14] "cycleGather.xml"                                                                                          
[15] "doc"                                                                                                      
[16] "executeActionList4IEM.py"                                                                                 
[17] "file.xml"                                                                                                 
[18] "fixedFixlets.txt"                                                                                         
[19] "getAssetsNCheckinTime.py"                                                                                 
[20] "getFixlets2DictUrllib3.py"                                                                                
[21] "getFixletsRest.py"                                                                                        
[22] "getFixletsRest2File.py"                                                                                   
[23] "getFixletsRest2File2.py"                                                                                  
[24] "getFixletsRestAdonix.py"                                                                                  
[25] "getFixletsRestAdonixUrllib.py"                                                                            
[26] "getFixletsRestGrass.py"                                                                                   
[27] "Grasskeet-CriticalVulns-baseline.bes"                                                                     
[28] "importTweedleDeeFixlet.py"                                                                                
[29] "importTweedleDeeFixlet3.py"                                                                               
[30] "invokeBesProvisioning.xml"                                                                                
[31] "InvokeFixletRestViaPython.docx"                                                                           
[32] "InvokeFixletRestViaPython.pdf"                                                                            
[33] "InvokeFixletRestViaPython2.pdf"                                                                           
[34] "Li3000.txt"                                                                                               
[35] "MasterSite_FixletA.bes"                                                                                   
[36] "MasterSite_FixletB.bes"                 

In [37]:
repl_python(); print("hello, there")

Python 3.6.4 (/anaconda3/bin/python)
Reticulate 1.9 REPL -- A Python interpreter in R.


exit
[1] "hello, there"


In [38]:
source_python("add.py")
add(4,16)

[1] 20

In [58]:
myCfg<-readConfig(rCfg)
newRelevance<-"names whose(it as lowercase contains %22adhay%22) of bes computers"
result<-queryViaRelevance(myCfg,newRelevance)
print(result)

[1] "character"
[1] "<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n<BESAPI xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"BESAPI.xsd\">\n\t<Query Resource=\"names whose(it as lowercase contains &quot;adhay&quot;) of bes computers\">\n\t\t<Result>\n\t\t\t<Answer type=\"string\">ADHAYGAP21</Answer>\n\t\t\t<Answer type=\"string\">ADHAYVCR01</Answer>\n\t\t\t<Answer type=\"string\">ADHAYFDB01</Answer>\n\t\t\t<Answer type=\"string\">ADHAYMCS01N1</Answer>\n\t\t\t<Answer type=\"string\">ADHAYGAP16</Answer>\n\t\t\t<Answer type=\"string\">ADHAYWEB08</Answer>\n\t\t\t<Answer type=\"string\">ADHAYFIM01</Answer>\n\t\t\t<Answer type=\"string\">ADHAYWEB34</Answer>\n\t\t\t<Answer type=\"string\">ADHAYDBP35</Answer>\n\t\t\t<Answer type=\"string\">ADHAYWEB16</Answer>\n\t\t\t<Answer type=\"string\">ADHAYWEB37</Answer>\n\t\t\t<Answer type=\"string\">ADHAYGAP03</Answer>\n\t\t\t<Answer type=\"string\">ADHAYGAP24</Answer>\n\t\t\t<Answer type=\"string\">ADHAYGAP73</Answer